In [ ]:
# ===============================
# Cell 1: Imports, Config, Device
# ===============================

# ---- Standard libraries ----
import os
import random
import math
import time
from glob import glob

# ---- Numerical & plotting ----
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- Image processing ----
import cv2
from PIL import Image

# ---- PyTorch ----
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# ---- Reproducibility ----
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ---- Device setup (Kaggle P100) ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# ---- Global project configuration ----
IMG_SIZE = 227          # As used in the paper
CHANNELS = 1            # Grayscale
SEQ_LEN = 10            # Sequence length
TEMPORAL_STRIDES = [1, 2, 3]  # Temporal data augmentation
BATCH_SIZE = 4          # Safe for ConvLSTM on P100
NUM_EPOCHS = 30
LEARNING_RATE = 1e-4

# ---- Dataset paths (Kaggle default) ----
DATA_ROOT = "/kaggle/input/avenue-dataset"
TRAIN_DIR = os.path.join(DATA_ROOT, "training_videos")
TEST_DIR = os.path.join(DATA_ROOT, "testing_videos")

# ---- Sanity print ----
print("Train dir:", TRAIN_DIR)
print("Test dir :", TEST_DIR)


In [ ]:
# ===============================
# Cell 2: Dataset Sanity Checks
# ===============================

def list_video_folders(root_dir):
    """Return sorted list of video folders"""
    if not os.path.exists(root_dir):
        raise FileNotFoundError(f"Directory not found: {root_dir}")
    return sorted([
        d for d in os.listdir(root_dir)
        if os.path.isdir(os.path.join(root_dir, d))
    ])

def count_frames(video_dir):
    """Count number of frame images in a video folder"""
    frames = glob(os.path.join(video_dir, "*.jpg"))
    return len(frames)

# ---- List training and testing videos ----
train_videos = list_video_folders(TRAIN_DIR)
test_videos = list_video_folders(TEST_DIR)

print(f"Number of training videos: {len(train_videos)}")
print(f"Number of testing videos : {len(test_videos)}")

# ---- Inspect a few videos ----
print("\nSample training videos and frame counts:")
for vid in train_videos[:3]:
    vid_path = os.path.join(TRAIN_DIR, vid)
    print(f"  Video {vid}: {count_frames(vid_path)} frames")

print("\nSample testing videos and frame counts:")
for vid in test_videos[:3]:
    vid_path = os.path.join(TEST_DIR, vid)
    print(f"  Video {vid}: {count_frames(vid_path)} frames")

# ---- Basic consistency checks ----
assert len(train_videos) > 0, "No training videos found!"
assert len(test_videos) > 0, "No testing videos found!"

# ---- Peek at frame naming ----
sample_vid = train_videos[0]
sample_frames = sorted(glob(os.path.join(TRAIN_DIR, sample_vid, "*.jpg")))

print("\nFirst 5 frame files in training video", sample_vid)
for f in sample_frames[:5]:
    print(" ", os.path.basename(f))


In [ ]:
# =====================================
# Cell 3: Image Preprocessing Utilities
# =====================================

def load_and_preprocess_image(
    img_path,
    mean_image=None,
    std_image=None
):
    """
    Load image, resize, grayscale, scale to [0,1],
    subtract global mean image, normalize to zero mean & unit variance.
    """
    # ---- Load image ----
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Failed to read image: {img_path}")

    # ---- Resize ----
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

    # ---- Convert to grayscale ----
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # ---- Scale to [0, 1] ----
    img = img.astype(np.float32) / 255.0

    # ---- Subtract global mean image ----
    if mean_image is not None:
        img = img - mean_image

    # ---- Normalize to zero mean & unit variance ----
    if std_image is not None:
        img = img / (std_image + 1e-8)

    # ---- Add channel dimension ----
    img = np.expand_dims(img, axis=0)  # (1, H, W)

    return img


def compute_global_mean_std(train_dir, max_frames_per_video=None):
    """
    Compute global mean image and std image
    over the entire training dataset.

    To keep it memory-safe, images are accumulated incrementally.
    """
    print("Computing global mean and std from training data...")

    sum_image = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.float64)
    sum_sq_image = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.float64)
    total_frames = 0

    video_folders = sorted(os.listdir(train_dir))

    for vid in video_folders:
        vid_path = os.path.join(train_dir, vid)
        frame_paths = sorted(glob(os.path.join(vid_path, "*.jpg")))

        if max_frames_per_video is not None:
            frame_paths = frame_paths[:max_frames_per_video]

        for fp in frame_paths:
            img = cv2.imread(fp)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = img.astype(np.float32) / 255.0

            sum_image += img
            sum_sq_image += img ** 2
            total_frames += 1

    mean_image = sum_image / total_frames
    var_image = (sum_sq_image / total_frames) - (mean_image ** 2)
    std_image = np.sqrt(np.maximum(var_image, 1e-8))

    print(f"Computed mean/std using {total_frames} frames")

    return mean_image.astype(np.float32), std_image.astype(np.float32)


In [ ]:
# ==========================================
# Cell 4: Temporal Sequence Index Generation
# ==========================================

def generate_sequences_for_video(
    frame_paths,
    seq_len=SEQ_LEN,
    strides=TEMPORAL_STRIDES
):
    """
    Generate temporal sequences (lists of frame indices)
    for a single video using multiple temporal strides.

    Returns:
        List of lists, where each sublist contains frame indices.
    """
    sequences = []
    num_frames = len(frame_paths)

    for stride in strides:
        max_start = num_frames - (seq_len - 1) * stride
        for start_idx in range(max_start):
            seq_indices = [
                start_idx + i * stride
                for i in range(seq_len)
            ]
            sequences.append(seq_indices)

    return sequences


def build_sequence_index(root_dir):
    """
    Build sequence index for all videos in a directory.

    Returns:
        List of dicts with:
        {
            'video': video_name,
            'frame_indices': [i1, i2, ..., iT]
        }
    """
    all_sequences = []

    video_folders = sorted(os.listdir(root_dir))

    for vid in video_folders:
        vid_path = os.path.join(root_dir, vid)
        frame_paths = sorted(glob(os.path.join(vid_path, "*.jpg")))

        if len(frame_paths) < SEQ_LEN:
            continue

        seqs = generate_sequences_for_video(
            frame_paths,
            seq_len=SEQ_LEN,
            strides=TEMPORAL_STRIDES
        )

        for seq in seqs:
            all_sequences.append({
                "video": vid,
                "frame_indices": seq
            })

    return all_sequences


# ---- Build training sequence index ----
train_sequences = build_sequence_index(TRAIN_DIR)

print(f"Total training sequences generated: {len(train_sequences)}")

# ---- Inspect a few sequences ----
for i in range(3):
    print(train_sequences[i])


In [ ]:
# ==========================================
# Cell 5: Spatial Encoder
# ==========================================

class SpatialEncoder(nn.Module):
    def __init__(self):
        super(SpatialEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(
                in_channels=CHANNELS,
                out_channels=128,
                kernel_size=11,
                stride=4,
                padding=0
            ),
            nn.ReLU(inplace=True),

            nn.Conv2d(
                in_channels=128,
                out_channels=64,
                kernel_size=5,
                stride=2,
                padding=0
            ),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        """
        x: (B, C, H, W)
        """
        return self.encoder(x)


In [ ]:
# ==========================================
# Cell 6A: ConvLSTM Cell
# ==========================================

class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size):
        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        padding = kernel_size // 2

        self.conv = nn.Conv2d(
            in_channels=input_dim + hidden_dim,
            out_channels=4 * hidden_dim,
            kernel_size=kernel_size,
            padding=padding
        )

    def forward(self, x, h_prev, c_prev):
        combined = torch.cat([x, h_prev], dim=1)
        conv_out = self.conv(combined)

        cc_i, cc_f, cc_o, cc_g = torch.chunk(conv_out, 4, dim=1)

        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c = f * c_prev + i * g
        h = o * torch.tanh(c)

        return h, c


In [ ]:
# ==========================================
# Cell 6B: ConvLSTM Stack
# ==========================================

class ConvLSTM(nn.Module):
    def __init__(self):
        super(ConvLSTM, self).__init__()

        self.cell1 = ConvLSTMCell(64, 64, 3)
        self.cell2 = ConvLSTMCell(64, 32, 3)
        self.cell3 = ConvLSTMCell(32, 64, 3)

    def forward(self, x):
        """
        x: (B, T, C, H, W)
        """
        B, T, C, H, W = x.size()

        h1 = torch.zeros(B, 64, H, W, device=x.device)
        c1 = torch.zeros_like(h1)

        h2 = torch.zeros(B, 32, H, W, device=x.device)
        c2 = torch.zeros_like(h2)

        h3 = torch.zeros(B, 64, H, W, device=x.device)
        c3 = torch.zeros_like(h3)

        outputs = []

        for t in range(T):
            h1, c1 = self.cell1(x[:, t], h1, c1)
            h2, c2 = self.cell2(h1, h2, c2)
            h3, c3 = self.cell3(h2, h3, c3)
            outputs.append(h3.unsqueeze(1))

        return torch.cat(outputs, dim=1)


In [ ]:
# ==========================================
# Cell 7: Spatial Decoder
# ==========================================

class SpatialDecoder(nn.Module):
    def __init__(self):
        super(SpatialDecoder, self).__init__()

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=64,
                out_channels=128,
                kernel_size=5,
                stride=2,
                padding=0
            ),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(
                in_channels=128,
                out_channels=CHANNELS,
                kernel_size=11,
                stride=4,
                padding=0
            )
        )

    def forward(self, x):
        """
        x: (B, C, H, W)
        """
        return self.decoder(x)


In [ ]:
# ==========================================
# Cell 8: Full Spatiotemporal Autoencoder
# ==========================================

class SpatioTemporalAutoEncoder(nn.Module):
    def __init__(self):
        super(SpatioTemporalAutoEncoder, self).__init__()

        self.spatial_encoder = SpatialEncoder()
        self.temporal_model = ConvLSTM()
        self.spatial_decoder = SpatialDecoder()

    def forward(self, x):
        """
        x: (B, T, C, H, W)
        """
        B, T, C, H, W = x.size()

        # ---- Encode each frame spatially ----
        encoded = []
        for t in range(T):
            feat = self.spatial_encoder(x[:, t])
            encoded.append(feat.unsqueeze(1))

        encoded = torch.cat(encoded, dim=1)  # (B, T, 64, 26, 26)

        # ---- Temporal encoding/decoding ----
        temporal_out = self.temporal_model(encoded)

        # ---- Decode each frame spatially ----
        decoded = []
        for t in range(T):
            frame = self.spatial_decoder(temporal_out[:, t])
            decoded.append(frame.unsqueeze(1))

        return torch.cat(decoded, dim=1)


In [ ]:
# ==========================================
# Cell 9: Model Initialization & Params
# ==========================================

model = SpatioTemporalAutoEncoder().to(device)

# ---- Parameter count ----
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

num_params = count_parameters(model)

print("Model initialized.")
print(f"Total trainable parameters: {num_params / 1e6:.2f} M")


In [ ]:
# ==========================================
# Cell 10: Loss Function & Optimizer
# ==========================================

criterion = nn.MSELoss(reduction="mean")

optimizer = optim.Adam(
    model.parameters(),
    lr=LEARNING_RATE
)

print("Loss and optimizer initialized.")


In [ ]:
# ==========================================
# Cell 11: One Training Epoch
# ==========================================

def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0

    for batch_idx, batch in enumerate(dataloader):
        # batch: (B, T, C, H, W)
        batch = batch.to(device)

        optimizer.zero_grad()

        # ---- Forward ----
        recon = model(batch)

        # ---- Reconstruction loss ----
        loss = criterion(recon, batch)

        # ---- Backprop ----
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx}/{len(dataloader)} - Loss: {loss.item():.6f}")

    epoch_loss = running_loss / len(dataloader)
    return epoch_loss


In [ ]:
# ==========================================
# Cell 12: Train Dataset & DataLoader Setup
# ==========================================

# ---- Compute global mean and std ----
mean_image, std_image = compute_global_mean_std(TRAIN_DIR)

print(f"Global mean image shape: {mean_image.shape}")
print(f"Global std image shape: {std_image.shape}")
print(f"Mean value range: [{mean_image.min():.3f}, {mean_image.max():.3f}]")
print(f"Std value range: [{std_image.min():.3f}, {std_image.max():.3f}]")


class TrainDataset(Dataset):
    """Dataset for training sequences"""
    def __init__(self, root_dir, mean_img, std_img):
        self.samples = []
        self.root_dir = root_dir
        self.mean_img = mean_img
        self.std_img = std_img

        videos = sorted(os.listdir(root_dir))

        for vid in videos:
            frame_paths = sorted(glob(os.path.join(root_dir, vid, "*.jpg")))
            
            if len(frame_paths) < SEQ_LEN:
                continue
            
            seqs = generate_sequences_for_video(frame_paths)

            for seq in seqs:
                self.samples.append((frame_paths, seq))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        frame_paths, seq = self.samples[idx]

        frames = []
        for i in seq:
            img = load_and_preprocess_image(
                frame_paths[i],
                self.mean_img,
                self.std_img
            )
            frames.append(img)

        # Shape: (T, C, H, W)
        frames = torch.tensor(np.stack(frames), dtype=torch.float32)
        return frames


# ---- Create training dataset ----
train_dataset = TrainDataset(TRAIN_DIR, mean_image, std_image)

# ---- Create training dataloader ----
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

print(f"\nTraining dataset size: {len(train_dataset)}")
print(f"Training batches per epoch: {len(train_loader)}")


In [ ]:
# ==========================================
# Cell 12: Full Training Loop
# ==========================================

CHECKPOINT_DIR = "./checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

for epoch in range(1, NUM_EPOCHS + 1):
    start_time = time.time()

    epoch_loss = train_one_epoch(
        model,
        train_loader,
        optimizer,
        criterion,
        device
    )

    elapsed = time.time() - start_time
    print(f"Epoch [{epoch}/{NUM_EPOCHS}] - Loss: {epoch_loss:.6f} - Time: {elapsed:.2f}s")

    if epoch % 5 == 0:
        ckpt_path = os.path.join(CHECKPOINT_DIR, f"model_epoch_{epoch}.pth")
        torch.save(model.state_dict(), ckpt_path)
        print(f"Checkpoint saved: {ckpt_path}")


In [ ]:
# ==========================================
# Cell 13: Test Dataset
# ==========================================

class TestDataset(Dataset):
    def __init__(self, root_dir, mean_img, std_img):
        self.samples = []
        self.root_dir = root_dir
        self.mean_img = mean_img
        self.std_img = std_img

        videos = sorted(os.listdir(root_dir))

        for vid in videos:
            frame_paths = sorted(glob(os.path.join(root_dir, vid, "*.jpg")))
            seqs = generate_sequences_for_video(frame_paths)

            for seq in seqs:
                self.samples.append((vid, frame_paths, seq))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        vid, frame_paths, seq = self.samples[idx]

        frames = []
        for i in seq:
            img = load_and_preprocess_image(
                frame_paths[i],
                self.mean_img,
                self.std_img
            )
            frames.append(img)

        frames = torch.tensor(np.stack(frames), dtype=torch.float32)
        return frames, vid, seq


In [ ]:
# ==========================================
# Cell 14: Inference & Euclidean Error
# ==========================================

model.eval()
frame_errors = {}

with torch.no_grad():
    for batch, vids, seqs in test_loader:
        batch = batch.to(device)
        recon = model(batch)

        # Euclidean distance per frame
        error = torch.sqrt(
            torch.sum((recon - batch) ** 2, dim=(2, 3, 4))
        )  # (B, T)

        error = error.cpu().numpy()

        for b in range(batch.size(0)):
            vid = vids[b]
            seq = seqs[b]

            if vid not in frame_errors:
                frame_errors[vid] = {}

            for t, frame_idx in enumerate(seq):
                frame_errors[vid].setdefault(frame_idx, []).append(error[b, t])


In [ ]:
# ==========================================
# Cell 15: Aggregate Frame Scores
# ==========================================

frame_scores = {}

for vid in frame_errors:
    frame_scores[vid] = {}
    for frame_idx, errs in frame_errors[vid].items():
        frame_scores[vid][frame_idx] = np.mean(errs)


In [ ]:
# ==========================================
# Cell 16: Regularity Score
# ==========================================

all_scores = []
for vid in frame_scores:
    all_scores.extend(frame_scores[vid].values())

min_s, max_s = min(all_scores), max(all_scores)

regularity_scores = {}

for vid in frame_scores:
    regularity_scores[vid] = {}
    for frame_idx, score in frame_scores[vid].items():
        norm_score = (score - min_s) / (max_s - min_s + 1e-8)
        regularity_scores[vid][frame_idx] = 1.0 - norm_score


In [ ]:
# ==========================================
# Cell 17: Persistence1D Event Detection
# ==========================================

TEMPORAL_WINDOW = 50  # frames
PERSISTENCE_THRESHOLD = 0.1  # Minimum persistence value to consider as anomaly

def persistence_1d(regularity_scores_dict, temporal_window=TEMPORAL_WINDOW):
    """
    Persistence1D algorithm for grouping local minima.
    
    Low regularity scores (< 0.5) indicate potential anomalies.
    We find connected components of frames with low scores
    within the temporal window, then compute persistence.
    
    Args:
        regularity_scores_dict: {frame_idx: regularity_score}
        temporal_window: Max frames apart to consider connected
    
    Returns:
        events: List of {
            'frames': [frame indices],
            'min_score': minimum score in group,
            'persistence': max persistence value,
            'mean_score': average score
        }
    """
    
    # ---- Find potential anomaly frames (low regularity) ----
    frames = sorted(regularity_scores_dict.keys())
    
    # Create a mapping of frame -> score
    score_map = {f: regularity_scores_dict[f] for f in frames}
    
    # ---- Build connected components via union-find ----
    # Frames with low regularity that are close in time belong together
    parent = {f: f for f in frames}
    
    def find(x):
        if parent[x] != x:
            parent[x] = find(parent[x])
        return parent[x]
    
    def union(x, y):
        px, py = find(x), find(y)
        if px != py:
            parent[px] = py
    
    # Connect frames within temporal window
    for i in range(len(frames) - 1):
        if frames[i + 1] - frames[i] <= temporal_window:
            union(frames[i], frames[i + 1])
    
    # ---- Group frames by component ----
    components = {}
    for f in frames:
        root = find(f)
        if root not in components:
            components[root] = []
        components[root].append(f)
    
    # ---- Compute persistence for each component ----
    events = []
    
    for root, component_frames in components.items():
        component_frames = sorted(component_frames)
        
        # Persistence: measure of how strong/persistent the anomaly is
        # Higher persistence = more consistent anomaly
        scores = [score_map[f] for f in component_frames]
        
        # Persistence metric: lower regularity = higher persistence
        min_score = min(scores)
        mean_score = np.mean(scores)
        persistence = 1.0 - mean_score  # Invert: low regularity → high persistence
        
        # ---- Span of the event ----
        time_span = component_frames[-1] - component_frames[0]
        
        events.append({
            'frames': component_frames,
            'min_score': min_score,
            'mean_score': mean_score,
            'persistence': persistence,
            'time_span': time_span,
            'num_frames': len(component_frames)
        })
    
    # Sort by persistence (strongest anomalies first)
    events.sort(key=lambda e: e['persistence'], reverse=True)
    
    return events


def filter_events_by_persistence(events, threshold=PERSISTENCE_THRESHOLD):
    """Filter events by persistence threshold."""
    return [e for e in events if e['persistence'] >= threshold]


# ---- Apply Persistence1D to each video ----
anomalous_events = {}

for vid in regularity_scores:
    events = persistence_1d(regularity_scores[vid], TEMPORAL_WINDOW)
    
    # Filter by persistence threshold
    significant_events = filter_events_by_persistence(events, PERSISTENCE_THRESHOLD)
    
    anomalous_events[vid] = significant_events
    
    print(f"\n{vid}:")
    print(f"  Total events detected: {len(events)}")
    print(f"  Significant events (persistence >= {PERSISTENCE_THRESHOLD}): {len(significant_events)}")
    
    for idx, event in enumerate(significant_events[:3]):  # Show top 3
        print(f"    Event {idx+1}: frames {event['frames'][0]}-{event['frames'][-1]}, "
              f"persistence={event['persistence']:.3f}, "
              f"mean_regularity={event['mean_score']:.3f}")


In [ ]:
# ==========================================
# Cell 18: CSV Output with Persistence1D Results
# ==========================================

# ---- Save frame-level anomaly scores ----
rows = []

for vid in sorted(regularity_scores):
    for frame_idx in sorted(regularity_scores[vid]):
        rows.append({
            "Id": f"{vid}_{frame_idx}",
            "Score": regularity_scores[vid][frame_idx]
        })

df_frames = pd.DataFrame(rows)
df_frames.to_csv("anomaly_scores.csv", index=False)

print("Frame-level scores saved as anomaly_scores.csv")
print(df_frames.head())

# ---- Save event-level detections ----
event_rows = []
event_id = 1

for vid in sorted(anomalous_events):
    for event in anomalous_events[vid]:
        event_rows.append({
            "Event_ID": event_id,
            "Video": vid,
            "Start_Frame": event['frames'][0],
            "End_Frame": event['frames'][-1],
            "Num_Frames": event['num_frames'],
            "Time_Span_Frames": event['time_span'],
            "Persistence": event['persistence'],
            "Mean_Regularity": event['mean_score'],
            "Min_Regularity": event['min_score']
        })
        event_id += 1

df_events = pd.DataFrame(event_rows)
df_events.to_csv("detected_events.csv", index=False)

print("\n\nEvent-level detections saved as detected_events.csv")
print(df_events.head(10))
print(f"\nTotal anomalous events detected: {len(df_events)}")


In [ ]:
# ==========================================
# Cell 19: Visualization of Persistence1D Results
# ==========================================

import matplotlib.pyplot as plt

# Visualize results for a few videos
n_videos_to_plot = min(3, len(anomalous_events))

for vid_idx, vid in enumerate(sorted(anomalous_events.keys())[:n_videos_to_plot]):
    fig, ax = plt.subplots(figsize=(14, 4))
    
    # Plot regularity scores
    frames = sorted(regularity_scores[vid].keys())
    scores = [regularity_scores[vid][f] for f in frames]
    
    ax.plot(frames, scores, 'b-', linewidth=1, label='Regularity Score', alpha=0.7)
    ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Anomaly Threshold')
    
    # Highlight detected events
    events = anomalous_events[vid]
    colors = plt.cm.Reds(np.linspace(0.3, 0.9, len(events)))
    
    for event_idx, event in enumerate(events):
        event_frames = event['frames']
        if event_frames:
            ax.axvspan(event_frames[0], event_frames[-1], 
                      alpha=0.2, color=colors[event_idx],
                      label=f"Event {event_idx+1} (persist={event['persistence']:.2f})")
    
    ax.set_xlabel('Frame Index', fontsize=11)
    ax.set_ylabel('Regularity Score (1=normal, 0=anomaly)', fontsize=11)
    ax.set_title(f'{vid} - Persistence1D Anomaly Detection\n'
                f'Window={TEMPORAL_WINDOW}f, Threshold={PERSISTENCE_THRESHOLD}',
                fontsize=12, fontweight='bold')
    ax.legend(loc='upper right', fontsize=9)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f"persistence1d_{vid}.png", dpi=100, bbox_inches='tight')
    plt.show()
    
    print(f"Saved plot: persistence1d_{vid}.png")

# ---- Summary statistics ----
print("\n" + "="*60)
print("PERSISTENCE1D SUMMARY STATISTICS")
print("="*60)

total_events = sum(len(anomalous_events[vid]) for vid in anomalous_events)
print(f"Total videos analyzed: {len(anomalous_events)}")
print(f"Total anomalous events detected: {total_events}")

all_persistences = []
all_spans = []

for vid in anomalous_events:
    for event in anomalous_events[vid]:
        all_persistences.append(event['persistence'])
        all_spans.append(event['time_span'])

if all_persistences:
    print(f"\nPersistence statistics:")
    print(f"  Mean: {np.mean(all_persistences):.3f}")
    print(f"  Median: {np.median(all_persistences):.3f}")
    print(f"  Min: {np.min(all_persistences):.3f}")
    print(f"  Max: {np.max(all_persistences):.3f}")
    
    print(f"\nEvent duration (frames):")
    print(f"  Mean: {np.mean(all_spans):.1f}")
    print(f"  Median: {np.median(all_spans):.1f}")
    print(f"  Min: {np.min(all_spans):.0f}")
    print(f"  Max: {np.max(all_spans):.0f}")
    print(f"  (~2-3 sec at 24-25 fps = 48-75 frames)")
